In [1]:
import torchvision
from torchvision import  transforms 
import torch
from torch import no_grad
import requests
import cv2
import numpy as np

In [2]:
def draw_box(pred_class, img, rect_th=2, text_size=0.5, text_th=2, download_image=False, img_name="img"):
    """
    draws box around each object 
    
    predicted_classes: a list where each element contains a tuple that corresponds to information about the different objects; Each element includes a tuple with the class name, probability of belonging to that class and the coordinates of the bounding box corresponding to the object 
    image : frozen surface 
   
    """
    image = (np.clip(cv2.cvtColor(np.clip(img.numpy().transpose((1, 2, 0)), 0, 1), cv2.COLOR_RGB2BGR), 0, 1) * 255).astype(np.uint8).copy()

    for predicted_class in pred_class:
      
        label=predicted_class[0]
        probability=predicted_class[1]
        box=predicted_class[2]
        t = round(box[0][0].tolist())
        l = round(box[0][1].tolist())
        r = round(box[1][0].tolist())
        b = round(box[1][1].tolist())

        # Giving brief information about rectange, class and probability.
        from colorama import Fore
        from colorama import Style
        print(f"\nLabel: {Fore.GREEN}{label}{Style.RESET_ALL}")
        print(f"Box coordinates: {t}, {l}, {r}, {b}")
        print(f"Probability: {probability}")

        # Drawing rectangle and adding text on the picture based on their class and size.
        cv2.rectangle(image, (t, l), (r, b), (0, 255, 0), rect_th)
        cv2.rectangle(image, (t, l), (t+110, l+17), (255, 255, 255), -1)
        cv2.putText(image, label, (t+10, l+12),  cv2.FONT_HERSHEY_SIMPLEX, 
                  text_size, (0,255,0), thickness=text_th)
        cv2.putText(image, label+": "+str(round(probability, 2)), 
                  (t+10, l+12),  cv2.FONT_HERSHEY_SIMPLEX, text_size, 
                  (0, 255, 0),thickness=text_th)

        # Plotting image
        image = np.array(image)
        plt.figure(figsize=(15, 10))
        plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
        if download_image:
            plt.savefig(f'{img_name}.png')
        else:
            pass
        plt.show()

        del(img)
        del(image)

In [3]:
def save_RAM(image_=False):
    global image, img, pred
    torch.cuda.empty_cache()
    del(img)
    del(pred)
    if image_:
        image.close()
        del(image)

In [4]:
model_ = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
model_.eval()

for name, param in model_.named_parameters():
    param.requires_grad = False
print("done")

C:\Users\HOME\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\HOME\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=FasterRCNN_ResNet50_FPN_Weights.COCO_V1`. You can also use `weights=FasterRCNN_ResNet50_FPN_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


done


In [5]:
def model(x):
    with torch.no_grad():
        yhat = model_(x)
    return yhat

In [6]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Input, Conv2D, Dense, Flatten, Dropout
from tensorflow.keras.layers import GlobalMaxPooling2D, MaxPooling2D
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.models import Model
from tensorflow.keras.models import load_model

pred_model=load_model('res_model.h5')

In [19]:
import serial
import time
port = 'COM5'  
baud_rate = 9600
arduino = serial.Serial(port, baud_rate)

In [20]:
cap = cv2.VideoCapture(1)  # 0 for the default camera, adjust if using a different camera

while True:
    ret, frame = cap.read()
    if not ret:
        break
    transform = transforms.Compose([transforms.ToTensor()])
    img = transform(frame)
    pred = model([img])
    try:
        bounding_box=pred[0]['boxes'][0].tolist()
        t, l, r, b = [round(x) for x in bounding_box]
        img_plot=(np.clip(cv2.cvtColor(np.clip(img.numpy().transpose((1, 2, 0)), 0, 1), cv2.COLOR_RGB2BGR), 0, 1) * 255).astype(np.uint8)
        img_plot=cv2.rectangle(img_plot, (t, l), (r, b), (0, 255, 0), 10)
        target_size = (224, 224)  # Replace with your model's input size
        resized_frame = cv2.resize(frame[l:b,t:r], target_size)
        image_array = np.expand_dims(resized_frame, axis=0)
        image_array = image_array / 255.0
        predictions = pred_model.predict(image_array)
        classes=['Non-Biodegradable','Organic','Recyclable']
        prediction_class=classes[np.argmax(predictions)]
        # cv2.putText(img_plot, prediction_class, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (36,255,12), 2)
        print("Prediction:", prediction_class)
        cv2.imshow('Detected Object', img_plot)

        arduino.write(prediction_class.encode())
        break
    except:
        print("")
    plt.show()
    # Show the current frame for reference
    cv2.imshow('Camera Feed', frame)

    # Break the loop on 'q' key press
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the capture and close windows
cap.release()
cv2.destroyAllWindows()


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 368ms/step
Prediction: Non-Biodegradable


In [18]:
arduino.close()